In [1]:
import torch
import pandas as pd
import numpy as np
import math

from pathlib import Path
from typing import Dict, List
from datasets import concatenate_datasets, load_dataset
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
from transformers import pipeline


2023-09-09 13:39:53.432065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
notebook_login()

In [3]:
!python prepare_messages.py --tg-history-path "data/result.json" --output-path "data/data.csv"

In [4]:
data = load_dataset("csv", data_files="data/data.csv", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/ninachely/.cache/huggingface/datasets/csv/default-4b7761e56a21b280/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [5]:
print(data)

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 121829
})


In [6]:
data = data.filter(lambda example: example["context_1"] != None)
data

Filter:   0%|          | 0/121829 [00:00<?, ? examples/s]

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 79105
})

In [8]:
FIRST_SPEAKER_TOKEN = '@@ПЕРВЫЙ@@'
SECOND_SPEAKER_TOKEN = '@@ВТОРОЙ@@'
CONTEXT_COLS = ['context_3', 'context_2', 'context_1']
RESPONSE_COL = ['response']
SEP = ' '


def convert_to_dialog(sample: Dict[str, str]) -> Dict[str, str]:
    result_dict = dict()
    dialog = ""
    for i in range(len(CONTEXT_COLS)):
        key = CONTEXT_COLS[i]
        if key in sample and sample[key] is not None:
            speaker_token = FIRST_SPEAKER_TOKEN if i % 2 == 0 else SECOND_SPEAKER_TOKEN
            dialog += speaker_token + SEP + sample[key] + SEP
    
    response_key = RESPONSE_COL[0]
    if response_key in sample and sample[response_key] is not None:
        dialog += SECOND_SPEAKER_TOKEN + SEP + sample[response_key]
    result_dict['text'] = dialog
    return result_dict

In [9]:
data = data.train_test_split(test_size=0.2, shuffle=True)

In [10]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# https://huggingface.co/docs/transformers/tasks/language_modeling

In [12]:
dialog_data = data.map(convert_to_dialog)

Map:   0%|          | 0/63284 [00:00<?, ? examples/s]

Map:   0%|          | 0/15821 [00:00<?, ? examples/s]

In [13]:
type(dialog_data["train"]["text"])

list

In [14]:
def preprocess_function(examples):
    return tokenizer(list(examples["text"]))

In [15]:
tokenized_data = dialog_data.map(
    preprocess_function,
    batched = True,
    num_proc=4,
    remove_columns=data["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/63284 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/15821 [00:00<?, ? examples/s]

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-medium').to(device)

KeyboardInterrupt: 

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
arguments = {
    'output_dir': 'my-dialog-model',  # path to save the model's checkpoints
    'per_device_train_batch_size': 16,  # batch size per GPU/CPU for training
    'gradient_accumulation_steps': 4,  # number of batches to accumulate gradient
    'max_steps': 500,  # total number of optimizer.step() calls
    'save_steps': 100,  # save every save_steps
    'eval_steps': 100,  # run evaluation every eval_steps
    'dataloader_num_workers': 0,  # number of workers for data loading (default: 0)
    'save_total_limit': 2,  # total number of checkpoints to save, delete older checkpoints when reached
}

In [ ]:
trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()

In [ ]:
prompt = "МФТИ или Вышка"

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [ ]:
model = AutoModelForCausalLM.from_pretrained("my-ruDialoGPT-medium-model/checkpoint-10000/")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

In [ ]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
response